In [15]:
import init_spark_env
import pyspark
import pandas as pd

In [16]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, avg, stddev, stddev_pop, isnan, lower, transform, udf, replace, monotonically_increasing_id
from pyspark.sql.types import IntegerType, StructField, StructType, DataType

In [17]:
spark = SparkSession.builder.master('local').enableHiveSupport().appName('test_cli').getOrCreate()

In [18]:
spark

# 1)

In [19]:
spark.sql("DROP TABLE IF EXISTS test.advertiser;")
spark.sql("DROP TABLE IF EXISTS test.sales;")
spark.sql("DROP TABLE IF EXISTS test.company;")

DataFrame[]

In [20]:
spark.sql("CREATE DATABASE IF NOT EXISTS test")

DataFrame[]

In [21]:
spark.sql('''
          CREATE TABLE IF NOT EXISTS test.sales (
          `SalesID` Int,
          `advertise_id` Int,
          `compaign` Int);
          ''')

24/12/12 15:29:30 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/12/12 15:29:30 WARN HiveMetaStore: Location: file:/home/alexandr/stydy/eda/sql_git/SQL/ETL/spark-warehouse/test.db/sales specified for non-external table:sales


DataFrame[]

In [22]:
spark.sql('INSERT INTO test.sales values(1, 1, 1);')
spark.sql('INSERT INTO test.sales values(2, 2, 2);')
spark.sql('INSERT INTO test.sales values(3, 3, 3);')

DataFrame[]

In [23]:
spark.sql('INSERT INTO test.sales values(4, 1, 1);')
spark.sql('INSERT INTO test.sales values(5, 2, 2);')
spark.sql('INSERT INTO test.sales values(6, 3, 3);')

DataFrame[]

In [24]:
spark.sql('INSERT INTO test.sales values(7, 1, 1);')
spark.sql('INSERT INTO test.sales values(8, 1, 1);')

DataFrame[]

In [25]:
spark.sql("CREATE TABLE IF NOT EXISTS test.advertiser (AdvertiserID Int,name String);")

24/12/12 15:29:32 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/12/12 15:29:32 WARN HiveMetaStore: Location: file:/home/alexandr/stydy/eda/sql_git/SQL/ETL/spark-warehouse/test.db/advertiser specified for non-external table:advertiser


DataFrame[]

In [26]:
spark.sql('INSERT INTO test.advertiser values(1, "advertiser_1");')
spark.sql('INSERT INTO test.advertiser values(2, "advertiser_2");')
spark.sql('INSERT INTO test.advertiser values(3, "advertiser_3");')

DataFrame[]

In [27]:
spark.sql("CREATE TABLE IF NOT EXISTS test.company (CompanyID Int,compaing_name String);")

24/12/12 15:29:33 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/12/12 15:29:33 WARN HiveMetaStore: Location: file:/home/alexandr/stydy/eda/sql_git/SQL/ETL/spark-warehouse/test.db/company specified for non-external table:company


DataFrame[]

In [28]:
spark.sql('INSERT INTO test.company values(1, "company_1");')
spark.sql('INSERT INTO test.company values(2, "company_2");')
spark.sql('INSERT INTO test.company values(3, "company_3");')

DataFrame[]

In [29]:
spark.sql("SELECT * FROM test.sales;").show(), spark.sql("SELECT * FROM test.advertiser;").show(), spark.sql("SELECT * FROM test.company;").show()

+-------+------------+--------+
|SalesID|advertise_id|compaign|
+-------+------------+--------+
|      1|           1|       1|
|      2|           2|       2|
|      8|           1|       1|
|      6|           3|       3|
|      7|           1|       1|
|      4|           1|       1|
|      3|           3|       3|
|      5|           2|       2|
+-------+------------+--------+

+------------+------------+
|AdvertiserID|        name|
+------------+------------+
|           3|advertiser_3|
|           2|advertiser_2|
|           1|advertiser_1|
+------------+------------+

+---------+-------------+
|CompanyID|compaing_name|
+---------+-------------+
|        3|    company_3|
|        1|    company_1|
|        2|    company_2|
+---------+-------------+



(None, None, None)

In [30]:
spark.sql("SELECT * FROM test.advertiser;").show()

+------------+------------+
|AdvertiserID|        name|
+------------+------------+
|           3|advertiser_3|
|           2|advertiser_2|
|           1|advertiser_1|
+------------+------------+



In [31]:
data_sales = spark.sql("SELECT * FROM test.sales;")
data_advertiser = spark.sql("SELECT * FROM test.advertiser;")
data_company = spark.sql("SELECT * FROM test.company;")

In [32]:
spark.sql("""
          SELECT a.AdvertiserID ,
          a.name, count(s.SalesID),
          count( s.compaign)
          FROM test.advertiser as a, test.sales as s 
          where s.advertise_id=a.AdvertiserID 
          group by a.name, a.AdvertiserID;
          """).createOrReplaceTempView("count_")

In [33]:
spark.sql("SELECT * FROM count_;").show()

+------------+------------+--------------+---------------+
|AdvertiserID|        name|count(SalesID)|count(compaign)|
+------------+------------+--------------+---------------+
|           1|advertiser_1|             4|              4|
|           2|advertiser_2|             2|              2|
|           3|advertiser_3|             2|              2|
+------------+------------+--------------+---------------+



In [34]:
spark.sql("DROP VIEW count_;")

DataFrame[]

In [35]:
spark.sql("CREATE TABLE IF NOT EXISTS test.denorm_advertiser (AdvertiserID Int,name String, count_sales Int, count_company Int);")
spark.sql("SELECT * FROM test.denorm_advertiser;").show()

+------------+----+-----------+-------------+
|AdvertiserID|name|count_sales|count_company|
+------------+----+-----------+-------------+
+------------+----+-----------+-------------+



24/12/12 15:29:35 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/12/12 15:29:35 WARN HiveMetaStore: Location: file:/home/alexandr/stydy/eda/sql_git/SQL/ETL/spark-warehouse/test.db/denorm_advertiser specified for non-external table:denorm_advertiser


# 2)

In [36]:
spark.sql('''
          CREATE TABLE IF NOT EXISTS test.client (
          `ClientID` Int,
          `costomer_name` String,
          `country_id` Int,
          `country_name` String
          );
          ''')
spark.sql("SELECT * FROM test.client;").show()

+--------+-------------+----------+------------+
|ClientID|costomer_name|country_id|country_name|
+--------+-------------+----------+------------+
+--------+-------------+----------+------------+



24/12/12 15:29:36 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/12/12 15:29:36 WARN HiveMetaStore: Location: file:/home/alexandr/stydy/eda/sql_git/SQL/ETL/spark-warehouse/test.db/client specified for non-external table:client


# 3)

In [37]:
spark.sql("""
          SELECT s.SalesID, s.advertise_id, a.name, s.compaign, c.compaing_name
          FROM test.sales as s, test.advertiser as a, test.company as c
          where s.advertise_id=a.AdvertiserID and s.compaign=c.CompanyID
          """).show()

+-------+------------+------------+--------+-------------+
|SalesID|advertise_id|        name|compaign|compaing_name|
+-------+------------+------------+--------+-------------+
|      1|           1|advertiser_1|       1|    company_1|
|      2|           2|advertiser_2|       2|    company_2|
|      8|           1|advertiser_1|       1|    company_1|
|      6|           3|advertiser_3|       3|    company_3|
|      7|           1|advertiser_1|       1|    company_1|
|      4|           1|advertiser_1|       1|    company_1|
|      3|           3|advertiser_3|       3|    company_3|
|      5|           2|advertiser_2|       2|    company_2|
+-------+------------+------------+--------+-------------+



In [38]:
spark.sql('''
          CREATE TABLE IF NOT EXISTS test.sales_full(
          SELECT s.SalesID, s.advertise_id, a.name, s.compaign, c.compaing_name
          FROM test.sales as s, test.advertiser as a, test.company as c
          where s.advertise_id=a.AdvertiserID and s.compaign=c.CompanyID
          );
          ''')

24/12/12 15:29:36 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/12/12 15:29:36 WARN HiveMetaStore: Location: file:/home/alexandr/stydy/eda/sql_git/SQL/ETL/spark-warehouse/test.db/sales_full specified for non-external table:sales_full


DataFrame[]

In [39]:
spark.sql('SELECT * FROM test.sales_full;').show()

+-------+------------+------------+--------+-------------+
|SalesID|advertise_id|        name|compaign|compaing_name|
+-------+------------+------------+--------+-------------+
|      8|           1|advertiser_1|       1|    company_1|
|      3|           3|advertiser_3|       3|    company_3|
|      4|           1|advertiser_1|       1|    company_1|
|      5|           2|advertiser_2|       2|    company_2|
|      2|           2|advertiser_2|       2|    company_2|
|      6|           3|advertiser_3|       3|    company_3|
|      1|           1|advertiser_1|       1|    company_1|
|      7|           1|advertiser_1|       1|    company_1|
+-------+------------+------------+--------+-------------+



In [40]:
spark.sql('DROP DATABASE IF EXISTS test CASCADE;')

24/12/12 15:29:37 WARN TxnHandler: Cannot perform cleanup since metastore table does not exist


DataFrame[]

In [41]:
spark.stop()